# Workshop Part 2: The "Brain" (The Agent)

In this notebook, we act as the **Host**. We will connect to our running MCP server and implement the **Evaluator-Optimizer** pattern.

**The Loop:**
1. **Generator:** LLM writes code.
2. **Evaluator:** We call the `lint_code` tool on the server.
3. **Optimizer:** If the linter fails, we feed the error back to the LLM and retry.

In [ ]:
import os
import asyncio
from openai import OpenAI # Or Anthropic/Generic Client
from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client

# Configure your LLM (using a mock for this demo if no key is present)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

SERVER_URL = "http://localhost:8000/sse"

In [ ]:
async def agent_loop(user_prompt):
    messages = [
        {"role": "system", "content": "You are a Python coding assistant. Output ONLY raw python code. No markdown."}
    ]
    messages.append({"role": "user", "content": user_prompt})

    # Connect to our MCP Server via SSE
    async with sse_client(SERVER_URL) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            
            # Step 0: Initialize
            await session.initialize()
            
            # Start the Optimization Loop (Max 3 retries)
            for attempt in range(3):
                print(f"\n--- Attempt {attempt + 1} ---")
                
                # Step 1: GENERATE
                response = client.chat.completions.create(
                    model="gpt-4o", 
                    messages=messages
                )
                generated_code = response.choices[0].message.content.strip()
                print(f"Generated Code:\n{generated_code}")
                
                # Step 2: EVALUATE (Call MCP Tool)
                # We ask the server to lint the code
                lint_result = await session.call_tool("lint_code", arguments={"code": generated_code})
                lint_output = lint_result.content[0].text
                
                if lint_output == "OK":
                    print("✅ Linter Passed!")
                    
                    # Step 3: EXECUTE (Optional Final Step)
                    exec_result = await session.call_tool("run_safe_python", arguments={"code": generated_code})
                    print(f"Execution Output:\n{exec_result.content[0].text}")
                    return generated_code
                
                else:
                    print(f"❌ Linter Failed: {lint_output}")
                    # Feed the error back to the LLM
                    messages.append({"role": "assistant", "content": generated_code})
                    messages.append({"role": "user", "content": f"Your code failed linting with this error: {lint_output}. Fix it."})
            
            print("Failed to optimize code after 3 attempts.")

In [ ]:
# Trigger the Agent
# We intentionally request a tricky task that might cause a syntax error first

task = "Write a script that prints the first 5 Fibonacci numbers but make a syntax error by forgetting a colon."

# Run the async loop
await agent_loop(task)